# TDE-v2 Dataset Cookbook

This cookbooks severs as the preparation for the TDE-v2 dataset for the Chat-Transform project. It refactored the original dataset from Yeye-He's [repo](https://github.com/Yeye-He/Transform-Data-by-Example/tree/master/Benchmark) into a fine-tuning `Alpaca` format that is compatible with the [LLaMA-Factory](https://github.com/hiyouga/LLaMA-Factory)


In [2]:
import os
import pandas as pd
import json

## Load and Check

In [36]:
data_directory = '../data/TDE-v2/'
data_frames = []

# Load all JSON files in the directory and convert to DataFrame
for subdir, _, files in os.walk(data_directory):
    for file in files:
        if file.endswith('.json'):
            file_path = os.path.join(subdir, file)
            with open(file_path, 'r') as f:
                df = pd.json_normalize(json.load(f),
                                    meta=[['context', 'input'], ['context', 'output']])
                
                test_fp = file_path.replace(data_directory, "")
                df['test_path'] = os.path.splitext(test_fp)[0]  # remove file extension
                data_frames.append(df)

# Combine all DataFrames into a single DataFrame
df_all = pd.concat(data_frames, ignore_index=True)

# split the function from field 'instruction'
df_all['function'] = df_all['instruction'].apply(lambda x: x.split(':')[0].strip())

# check the shape of the dataframe
df_all.shape

(226, 7)

In [37]:
df_all.head()

,chat,instruction,tuples,context.input,context.output,test_path,function
0,Transform first and last names into username,format(): Combine first letter of first name w...,"[{'input': 'john smith', 'output': 'jsmith'}, ...",First and last names separated by a tab charac...,Username created by combining the first letter...,benchmark-FF-Trifacta-GoogleRefine/example_fil...,format()
1,Extract name of actor/actress,extract(): extract name of actor/actress from ...,[{'input': '* '''1953 [[Meena Kumari]] – ''[[B...,text format containing movie details,extracted name of actor/actress,benchmark-FF-Trifacta-GoogleRefine/example_fil...,extract()
2,Extract character names from wiki-style list,extract(): Extract the character name from the...,[{'input': '* '''1953 [[Meena Kumari]] – ''[[B...,Wiki-style list item containing actor and char...,Character name extracted from the input,benchmark-FF-Trifacta-GoogleRefine/example_fil...,extract()
3,format date of birth,format(): convert YYYYMMDD to MM-DD-YYYY,"[{'input': '19610223', 'output': '02-23-1961'}...",date in YYYYMMDD format,formatted date in MM-DD-YYYY,benchmark-FF-Trifacta-GoogleRefine/example_fil...,format()
4,Extract and format movie titles,transform(): Extract the movie title from the ...,[{'input': '* '''1953 [[Meena Kumari]] – ''[[B...,String containing movie information with title...,Lowercase movie title,benchmark-FF-Trifacta-GoogleRefine/example_fil...,transform()


> TDE-v2 dataset Data Dictionary

- The `chat` field represents a conversation between a user and an AI assistant. It is expected to be the input to the fine-tuned LM.

- The `instruction` field represents a coding instruction with API-style `function` name that specifies the transformation type. It is expected to be the target output from the fine-tuned LM.

- The `tuples` field represents a list of transformation-pairs, which serves as additional few-shot examples to help the fine-tuned LM to understand the user's intention. Each transformation-pair is a list of tuples, where each tuple contains an input and an output. The context of transformation pairs is stored in `context.input` and `context.output` fields, respectively.

- `test_path` refers to the original path of the test case in the TDE dataset.



In [40]:
# check duplicates of instruction, and sort it
df_all[df_all.duplicated(subset=['instruction'], keep=False)].sort_values(by='instruction')


,chat,instruction,tuples,context.input,context.output,test_path,function


In [41]:
# show distribution of function
df_all['function'].value_counts()


extract()             57
unit_convert()        48
format()              45
domain_calculate()    34
transform()           25
domain_map()          17
Name: function, dtype: int64

In [42]:
# count the number of transformation-pairs in tuples
df_all['num_tuples'] = df_all['tuples'].apply(len)
print(df_all['num_tuples'].describe()) 

print("Total number of tuples: ", df_all['num_tuples'].sum())
# show 

count    226.000000
mean       5.575221
std        2.232309
min        4.000000
25%        5.000000
50%        5.000000
75%        5.000000
max       25.000000
Name: num_tuples, dtype: float64
Total number of tuples:  1260


## Refactor to Alpaca Format

The fine-tuned LM serves as an AI assistant to translate user chat into the target instruction (chat-to-instruction), with the help of transformation-pairs stored in `tuples` field and the context information stored in `context.input` and `context.output` fields.

In [43]:
df_all.columns

Index(['chat', 'instruction', 'tuples', 'context.input', 'context.output',
       'test_path', 'function', 'num_tuples'],
      dtype='object')

### Supervised Fine-Tuning Dataset

Here we craft the fine-tuning dataset in Alpaca format. Regarding the above dataset, the *dataset description* in `dataset_info.json` of `LLaMA-Factory` should be the following 4 options:

1. `tde-o1`: chat-only

```json
"tde-o1": {
  "file_name": "tde-alpaca.json",
  "columns": {
    "prompt": "chat", // user chat
    "query": "", // no human input
    "response": "instruction", // model response
    "system": ...
  }
}
```

2. `tde-o2`: chat + transformation-pairs

```json
"tde-o2": {
  "file_name": "tde-alpaca.json",
  "columns": {
    "prompt": "chat", // user chat
    "query": "pairs", // provide transformation-pairs
    "response": "instruction", // model response
    "system": ...
  }
}
```

3. `tde-o3`: chat + context

```json
"tde-o3": {
  "file_name": "tde-alpaca.json",
  "columns": {
    "prompt": "chat", // user chat
    "query": "ctx", // provide context
    "response": "instruction", // model response
    "system": ...
  }
}
```

4. `tde-o4`: chat + context + transformation-pairs

```json
"tde-o4": {
  "file_name": "tde-alpaca.json",
  "columns": {
    "prompt": "chat", // user chat
    "query": "ctx_pairs", // provide context + transformation-pairs
    "response": "instruction", // model response
    "system": ...
  }
}
```



In [44]:
# show a sample of the dataframe
df_all.iloc[0]


chat                   Transform first and last names into username
instruction       format(): Combine first letter of first name w...
tuples            [{'input': 'john	smith', 'output': 'jsmith'}, ...
context.input     First and last names separated by a tab charac...
context.output    Username created by combining the first letter...
test_path         benchmark-FF-Trifacta-GoogleRefine/example_fil...
function                                                   format()
num_tuples                                                        5
Name: 0, dtype: object

The `chat` and `instruction` fields remain the same. We will refactor the `tuples` field into `io_pairs` field, and add `context.*` field to store the context information.

In [57]:
# refactor columns based on Alpaca format
df_alpaca = df_all.copy()

# set system prompt
df_alpaca['system'] = """You are an API assistant. When a user gives you a task, respond by selecting the most appropriate function from the following list: extract(), format(), unit_convert(), domain_map(), and domain_calculate(). 
If the user asks for functionality outside of these functions, you can use transform() by default. 

Always format your response like this:
[function_name]: [brief description of how the function works]

You should not invent new functions or modify the existing ones."""

# context information
df_alpaca['ctx'] = df_alpaca.apply(lambda x: 
                                   'Context information about the transformation input and output:\n' +
                                   'Input: ' + x['context.input'] + '\n' + 
                                   'Output: ' + x['context.output'],axis=1)

# transformation pairs
df_alpaca['pairs'] = df_alpaca['tuples'].apply(lambda x: 
                                               'Here are some transformation pairs:\n' +
                                               '\n'.join([f'Input: {t["input"]}\nOutput: {t["output"]}' for t in x])) 


# ctx + pairs
df_alpaca['ctx_pairs'] = df_alpaca['ctx'] + '\n\n' + df_alpaca['pairs']

# drop the source columns
df_alpaca.drop(columns=['context.input', 'context.output', 'tuples', 'function', 'num_tuples'], inplace=True)


# re-order columns
df_alpaca = df_alpaca[['test_path', 'system', 'chat', 'instruction', 'ctx', 'pairs', 'ctx_pairs']]


df_alpaca.head()


,test_path,system,chat,instruction,ctx,pairs,ctx_pairs
0,benchmark-FF-Trifacta-GoogleRefine/example_fil...,You are an API assistant. When a user gives yo...,Transform first and last names into username,format(): Combine first letter of first name w...,Context information about the transformation i...,Here are some transformation pairs:\nInput: jo...,Context information about the transformation i...
1,benchmark-FF-Trifacta-GoogleRefine/example_fil...,You are an API assistant. When a user gives yo...,Extract name of actor/actress,extract(): extract name of actor/actress from ...,Context information about the transformation i...,Here are some transformation pairs:\nInput: * ...,Context information about the transformation i...
2,benchmark-FF-Trifacta-GoogleRefine/example_fil...,You are an API assistant. When a user gives yo...,Extract character names from wiki-style list,extract(): Extract the character name from the...,Context information about the transformation i...,Here are some transformation pairs:\nInput: * ...,Context information about the transformation i...
3,benchmark-FF-Trifacta-GoogleRefine/example_fil...,You are an API assistant. When a user gives yo...,format date of birth,format(): convert YYYYMMDD to MM-DD-YYYY,Context information about the transformation i...,Here are some transformation pairs:\nInput: 19...,Context information about the transformation i...
4,benchmark-FF-Trifacta-GoogleRefine/example_fil...,You are an API assistant. When a user gives yo...,Extract and format movie titles,transform(): Extract the movie title from the ...,Context information about the transformation i...,Here are some transformation pairs:\nInput: * ...,Context information about the transformation i...


In [58]:
# Print all details of the first row in df_alpaca
print("Chat:")
print(df_alpaca.iloc[0]['chat'])
print("\nInstruction:")
print(df_alpaca.iloc[0]['instruction'])
print("\nTest Path:")
print(df_alpaca.iloc[0]['test_path'])
print("\nSystem:")
print(df_alpaca.iloc[0]['system'])
print("\nContext and Pairs:")
print(df_alpaca.iloc[0]['ctx_pairs'])

Chat:
Transform first and last names into username

Instruction:
format(): Combine first letter of first name with full last name to create username

Test Path:
benchmark-FF-Trifacta-GoogleRefine/example_file_name

System:
You are an API assistant. When a user gives you a task, respond by selecting the most appropriate function from the following list: extract(), format(), unit_convert(), domain_map(), and domain_calculate(). 
If the user asks for functionality outside of these functions, you can use transform() by default. 

Always format your response like this:
[function_name]: [brief description of how the function works]

You should not invent new functions or modify the existing ones.

Context and Pairs:
Context information about the transformation input and output:
Input: First and last names separated by a tab character.
Output: Username created by combining the first letter of the first name with the full last name.

Here are some transformation pairs:
Input: john	smith
Output

In [59]:
# export to JSON
df_alpaca.to_json('../data/TDE-alpaca/tde-alpaca.json', orient='records', indent=4, force_ascii=False)